In [5]:
import numpy as np
from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.circuit import Parameter
from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ

from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ

# Load IBMQ account
device = IBMQ.get_backend('ibmq_16_melbourne')
# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')

properties = device.properties()
coupling_map = device.configuration().coupling_map

shots = 8192 
max_credits = 3 


In [6]:
# List of gate times for ibmq_14_melbourne device
# Note that the None parameter for u1, u2, u3 is because gate
# times are the same for all qubits
gate_times = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678), ('cx', [1, 2], 547), ('cx', [2, 3], 721),
    ('cx', [4, 3], 733), ('cx', [4, 10], 721), ('cx', [5, 4], 800),
    ('cx', [5, 6], 800), ('cx', [5, 9], 895), ('cx', [6, 8], 895),
    ('cx', [7, 8], 640), ('cx', [9, 8], 895), ('cx', [9, 10], 800),
    ('cx', [11, 10], 721), ('cx', [11, 3], 634), ('cx', [12, 2], 773),
    ('cx', [13, 1], 2286), ('cx', [13, 12], 1504), ('cx', [], 800)
]

# Construct the noise model from backend properties
# and custom gate times
noise_model = noise.device.basic_device_noise_model(properties, gate_times=gate_times)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'u2', 'u3']
  Instructions with noise: ['u2', 'u3', 'cx', 'measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
  Specific qubit errors: [('u2', [0]), ('u2', [1]), ('u2', [2]), ('u2', [3]), ('u2', [4]), ('u2', [5]), ('u2', [6]), ('u2', [7]), ('u2', [8]), ('u2', [9]), ('u2', [10]), ('u2', [11]), ('u2', [12]), ('u2', [13]), ('u3', [0]), ('u3', [1]), ('u3', [2]), ('u3', [3]), ('u3', [4]), ('u3', [5]), ('u3', [6]), ('u3', [7]), ('u3', [8]), ('u3', [9]), ('u3', [10]), ('u3', [11]), ('u3', [12]), ('u3', [13]), ('cx', [1, 0]), ('cx', [1, 2]), ('cx', [2, 3]), ('cx', [4, 3]), ('cx', [4, 10]), ('cx', [5, 4]), ('cx', [5, 6]), ('cx', [5, 9]), ('cx', [6, 8]), ('cx', [7, 8]), ('cx', [9, 8]), ('cx', [9, 10]), ('cx', [11, 3]), ('cx', [11, 10]), ('cx', [11, 12]), ('cx', [12, 2]), ('cx', [13, 1]), ('cx', [13, 12]), ('measure', [0]), ('measure', [1]), ('measure', [2]), ('measure', [3]), ('measure', [4]), ('measure', [5]), ('measure', [6]), 

In [7]:
# System Specification
n = 3
# System Specification
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n
circ = QuantumCircuit(q,c)
phi = Parameter('phi')
# Test Circuit
circ.h(q[0])
for i in range(n-1):
    circ.cx(i, i+1)  
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()
for i in reversed(range(n-1)):
    circ.cx(i, i+1)
circ.h(q[0])
circ.barrier()
for i in range(n):
    circ.measure(q[i], c[i]) 
#circ.draw(output='mpl')
print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 


# Ideal execution
job = execute(circ,backend=simulator, parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots)

result_ideal = job.result()

counts_ideal =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_ideal)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_ideal)))


Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Counts
[{'000': 8192}, {'001': 6998, '000': 1194}, {'001': 4131, '000': 4061}, {'001': 1142, '000': 7050}, {'001': 8192}, {'001': 1246, '000': 6946}, {'001': 4110, '000': 4082}, {'001': 6957, '000': 1235}]
Counts of 000
[1.0, 0.145751953125, 0.4957275390625, 0.860595703125, 0.0, 0.847900390625, 0.498291015625, 0.1507568359375]


In [10]:
# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates


job = execute(circ, backend=simulator,
              basis_gates=basis_gates,
              coupling_map=coupling_map,
              noise_model=noise_model, parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots )

result_noise = job.result()

counts_noise =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_noise)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_noise)))


Counts
[{'100': 616, '000': 6686, '010': 376, '001': 277, '101': 61, '110': 49, '011': 87, '111': 40}, {'100': 124, '000': 1268, '010': 209, '001': 5678, '101': 565, '110': 49, '011': 259, '111': 40}, {'100': 339, '000': 3439, '010': 249, '001': 3444, '101': 370, '110': 49, '011': 257, '111': 45}, {'100': 565, '000': 5753, '010': 263, '001': 1156, '101': 156, '110': 57, '011': 206, '111': 36}, {'100': 60, '000': 260, '010': 65, '001': 6714, '101': 616, '110': 34, '011': 384, '111': 59}, {'100': 557, '000': 5758, '010': 261, '001': 1186, '101': 161, '110': 55, '011': 176, '111': 38}, {'100': 345, '000': 3446, '010': 217, '001': 3500, '101': 367, '110': 48, '011': 226, '111': 43}, {'100': 141, '000': 1220, '010': 204, '001': 5782, '101': 502, '110': 34, '011': 263, '111': 46}]
Counts of 000
[0.816162109375, 0.15478515625, 0.4197998046875, 0.7022705078125, 0.03173828125, 0.702880859375, 0.420654296875, 0.14892578125]
